# Segmenting and Clustering Neighborhoods

First we need to import the required libraries for the  project.

In [1]:
#importing required libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

### Import Data
1. I retrieved the data from the source website using the requests library.
2. Use Beautiful Soup to find instances of 'table' on the website.
3. Use pandas to convert it into a dataframe.

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' #identify url
source = requests.get(url).text #scrape the website
soup = BeautifulSoup(source, 'lxml') #parse the code
table = soup.find_all('table') #find instances of table
df = pd.read_html(str(table))[0] #convert to data frame

### Identify  Missing Values
Before cleaning the data had 180 rows.
After using `.describe()` I could see that there were 180 unique values for Postal Code but  77 "Not Assigned" Boroughs.

In [3]:
df.describe()

,Postal code,Borough,Neighborhood
count,180,180,103
unique,180,11,98
top,M7X,Not assigned,Downsview
freq,1,77,4


### Deal with missing values
After removing those I could see that I now had 103 rows.

In [4]:
print("Before Cleaning",df.shape)
df = df[df['Borough']!="Not assigned"] #Remove Not Assigned from Borough
print("Removed Not Assigned",df.shape)

Before Cleaning (180, 3)
Removed Not Assigned (103, 3)


The assignment instructions stated:
>If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

However, when I tried to find any unassigned there weren't any!

In [5]:
df[['Neighborhood']].isna().groupby('Neighborhood').count()

""
Neighborhood
False


In [6]:
#Commented out as the code is redundant. There is no data that meets those requirements for cleaning
    #df = df[df['Neighborhood']!=np.nan]
    #print("Removed Nulls Cleaning",df.shape)

### Correct the data format
Next, I reformatted the Neighborhood column so that the " /" were replaced with ",".\
I then capitalised the "c" in Postal code.
I then re-indexed to reflect the missing columns.

In [7]:
df['Neighborhood']  = df['Neighborhood'].str.replace(" /", ",")  # replaced / with ,
df.columns=["Postal Code", "Borough", "Neighborhood"]
df.reset_index(drop=True,inplace=True)
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Commentary
Whilst there are expected duplicates in the borough column, I can see there are no duplicates in the postal code column.\
There are however, duplicates in the Neighborhood column.

In [8]:
df.describe(include='all')

,Postal Code,Borough,Neighborhood
count,103,103,103
unique,103,10,98
top,M5X,North York,Downsview
freq,1,24,4


### Assumptions
I will assume that those duplicates are due to a Neighborhood spanning multiple postcodes but only **a single borough per Neighborhood**.\
The code below also shows that there are no hidden duplicates of Borough, through different spellings or formats.

In [9]:
df['Borough'].value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East Toronto         5
York                 5
East York            5
Mississauga          1
Name: Borough, dtype: int64

#### Let's see  what the final data frame looks like.

In [10]:
df.shape

(103, 3)

# Importing Geospatial Data

Data imported from csv directly into data frame.

In [11]:
geo = pd.read_csv("https://cocl.us/Geospatial_data")
geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Next, we need to do a left join of df and geo dataframes.

In [12]:
df = df.merge(geo,on="Postal Code")
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
